In [ ]:
import datetime
from plotly import graph_objects as go
from plotly.graph_objects import Layout
import pandas as pd
import numpy as np
import datetime

In [ ]:
path = "./weight_loss_dfs/jordan_df.pqt"
df = pd.read_parquet(path)

In [ ]:
class FigureGenerator():
    def __init__(self, df):
        self.modes = ["start_delta", "end_delta"]
        self.df = df
        self.columns = df.columns
        
    def create_scatter(self, axes, start=None, end=None, mode=None, layout=None):
        df = self.df
        
        for axis in axes:
            if axis not in df.columns:
                raise ValueError(
                    "{} is not a column in the dataframe. All axes must be columns in the dataframe.".format(
                        axis
                    )
                )
        
        if start is not None:
            if not isinstance(start, datetime.date):
                raise ValueError("start must be an instance of datetime.date")
            else:
                df = df[df.index >= str(start)]
        else:
            start = df.index[0]
                
        if end is not None:
            if not isinstance(end, datetime.date):
                raise ValueError("end must be an instance of datetime.date")
            else:
                df = df[df.index <= str(end)]
        else:
            end = df.index[-1]
            
        if mode is not None:
            if mode not in self.modes:
                raise ValueError("{} mode is invalid".format(mode))
        
        traces = self.get_traces(df, axes, mode=mode)
        fig = go.Figure(
            data=traces,
            layout=layout
        )
        
        return fig
    
    def create_heat_map(self, axes, start=None, end=None, mode=None):
        pass
        
    def get_traces(self, df, axes, mode):
        # get axis range here? leave it up to plotly for now
        traces = []
        for axis in axes:
            axis_df = df.copy()
            if len(axis_df[axis].dropna()) > 0:
                if mode is not None:
                    if mode == "start_delta":
                        axis_df[axis] = axis_df[axis] - axis_df[axis].dropna()[-1]
                    elif mode == "end_delta":
                        axis_df[axis] = axis_df[axis] - axis_df[axis].dropna()[0]
                    else:
                        pass
                else:
                    pass
                traces.append(
                    go.Scatter(
                        x=axis_df.index,
                        y=axis_df[axis],
                        name=axis,
                        showlegend=True,
                        connectgaps=True
                    )
                )
            else:
                pass
        return traces

In [ ]:
figure_generator = FigureGenerator(df=df)

In [ ]:
class FigureSelector:
    def __init__(self, figure_generator, recipe):
        self.figure_generator = figure_generator
        self.recipe = recipe
        
        self.items_by_unit = {}
        self.items_by_plot_type = {}
        for item in self.recipe:
            li = self.items_by_unit.get(item["units"], None)
            if li is None:
                li = []
                self.items_by_unit[item["units"]] = li
            li.append(item["col_title"])
            
            li = self.items_by_plot_type.get(item["plot_type"], None)
            if li is None:
                li = []
                self.items_by_plot_type[item["plot_type"]] = li
            li.append(item["col_title"])

In [ ]:
recipe = [
    {
        "col_title": "Weight",
        "plot_type": "scatter",
        "units": "lb",
    },
    {
        "col_title": "Workout",
        "plot_type": "heatmap",
        "units": "bool",
    },
    {
        "col_title": "Cardio",
        "plot_type": "heatmap",
        "units": "bool",
    }
]

In [ ]:
fs = FigureSelector(figure_generator=figure_generator, recipe=recipe)

In [ ]:
def gen_heatmap(df, columns, interval, start_date=None, end_date=None, ignore_zero=False, layout=None, colorscale="Oranges"):
    if start_date is None:
        dt = datetime.datetime.strptime(df.iloc[-1].name, "%Y-%m-%d")
        prev_date = datetime.date(day = dt.day, month=dt.month, year=dt.year)
    else:
        prev_date = start_date
    if end_date is None:
        dt = datetime.datetime.strptime(df.iloc[0].name, "%Y-%m-%d")
        last_date = datetime.date(day = dt.day, month=dt.month, year=dt.year)
    else:
        last_date = end_date
        
    curr_date = prev_date 
    
    interval_counts = [[] for col in columns]
    dates = []
    
    # NTS something wrong in the date-setting logic here. Values don't match dates
    while prev_date <= last_date:
        if interval == "weekly":
            curr_date = curr_date + datetime.timedelta(days=7)
        elif interval == "daily":
            curr_date = curr_date + datetime.timedelta(days=1)
        else:
            raise ValueError("not good!")    
        
        if interval == "daily":
            dates.append(f"{str(prev_date)}")
        else:
            dates.append(f"{str(prev_date)} - {str(curr_date - datetime.timedelta(days=1))}")
        
            
        df_curr = df[(df.index >= str(prev_date)) & (df.index < str(curr_date))][columns].dropna()
        
        for i, col in enumerate(columns):
            total = sum(df_curr[col])
            
            if ignore_zero and total == 0:
                total = np.nan
                
            interval_counts[i].append(total)
            

        prev_date = curr_date
            
    fig = go.Figure(
        data=go.Heatmap(
            z=interval_counts,
            x=dates,
            y=columns,
            colorscale=colorscale,
        ),
        layout=layout
    )
    
    display(fig)


In [ ]:
gen_heatmap(
    df,
    interval="weekly",
    columns=["Workout", "Cardio", "Meditate", "Stretch"],
    layout=Layout(
        title="Frequency of Activities, by Week",
    ),
    colorscale="Greens",
    #start_date=datetime.date(month=8, day=25, year=2022)
)

In [ ]:
df["Net Calories (kcal)"] = df["Consumed Calories (kcal)"] - df["Active Calories (kcal)"]
#display(df[[
#    "Net Calories (kcal)", "Consumed Calories (kcal)", "Active Calories (kcal)"
#]])
gen_heatmap(
    df,
    interval="daily",
    columns=["Net Calories (kcal)"],
    ignore_zero=True,
    layout=Layout(
        title="Net Calories, by Week",
    ),
    #start_date=datetime.date(month=8, day=25, year=2022),
    colorscale="OrRd",
)

In [ ]:
figure_generator.create_scatter(
    axes=[
        "Weight (lb)",
        #"Skeletal Muscle Mass (lb)",
        #"Waist (in)",
        #"Belly (in)",
        #"Hips (in)",
        "BMI",
        "Body Fat (%)",
    ],
    mode="start_delta",
    layout=Layout(
        title="Smaller is Better Values"
    ),
    #start=datetime.date(month=8, day=31, year=2022)
)

In [ ]:
figure_generator.create_scatter(
    axes=[
        #"Weight (lb)",
        "Waist (in)",
        "Belly (in)",
        "Hips (in)",
        #"BMI",
        #"Body Fat (%)",
    ],
    mode="start_delta",
    layout=Layout(
        title="Smaller is Better Values"
    ),
    #start=datetime.date(month=8, day=31, year=2022)
)

In [ ]:
figure_generator.create_scatter(
    axes=[
        "Bicep (in)",
        "Thigh (in)",
        "Chest (in)",
        "Calf (in)",
    ],
    mode="start_delta",
    layout=Layout(
        title="Larger is better"
    ),
    #start=datetime.date(month=8, day=31, year=2022)
    #start=datetime.date.today()-datetime.timedelta(days=7)
)

In [ ]:
figure_generator.create_scatter(
    axes=[
        #"Weight (lb)",
        "Skeletal Muscle Mass (lb)",
        "Bone Mass (lb)",
        "Body Fat (%)"
    ],
    mode="start_delta",
    layout=Layout(
        title="What am I Made of?"
    ),
    #start=datetime.date(month=8, day=31, year=2022),
)

In [ ]:
figure_generator.create_scatter(
    axes=[
        "Mile Time (min)",
    ],
    #mode="start_delta",
    layout=Layout(
        title="How fast am I?"
    )
)

In [ ]:
figure_generator.create_scatter(
    axes=[
        "Resting Heart Rate (bpm)", "Systolic BP (mmHg)", "Diastolic BP (mmHg)"
    ],
    #mode="start_delta",
    layout=Layout(
        title="Resting Heart Rate"
    ),
    #start=datetime.date(month=8, day=31, year=2021),
)

In [ ]:
df.sort_index(ascending=False)